### Model Cars Store Database: Exploring KPIs using SQL

In this guided project, I examine a database from a store that sells model products (e.g. model cars, planes). 

I want to answer **three main questions** by drawing insights from this database:
1. Which products should we order more of or less of?
2. How should we match marketing and communication strategies to customer behaviors?
3. How much can we spend on acquiring new customers?

**To answer the first question**, I looked at the top performing items that were low in stock. The performance of each item is determined by its total revenue. The store should prioritise the orders of items that are high performing and are running low in stock, while deprioritising those that are low performing. The top 3 performing items that were low in stock were: 2002 Suzuki XREO, 1968 Dodge Charger, 1965 Aston Martin DB5.

**Second**, to see how I can help the store tailor its communication by customer behaviour, I looked into the profit generated by each customer. This allowed me to highlight the “VIPs” – those that generate the highest profits for the store. Two of the five VIPs identified are from the USA, one from Spain, one from Australia, and one from France, so the VIPs are situated around the world, meaning that any marketing efforts aimed to engage with these loyal customers should be global in nature.

**Third**, I identified how much the store could spend in acquiring new customers by looking into the average profit per customer. Each customer approximately 39,039USD in profits — so in terms of acquisition, the store could spend up to a 70-80% of this figure (after discounting to get the net present value of the profit figure).

Note: This notebook is based off of a guided project that is part of the DataQuest "SQL Fundamentals". More information on this path can be found here: https://app.dataquest.io/m/600/guided-project%3A-customers-and-products-analysis-using-sql/2/introduction-to-our-project

---
### Importing SQL lite and connecting to database



I integrate SQL into this Python Notebook to explore key APIs on customers and products.
- *Blog post that explains how to integrate SQL in Jupyter https://towardsdatascience.com/heres-how-to-run-sql-in-jupyter-notebooks-f26eb90f3259*

In [10]:
#!pip install ipython-sql
import sqlalchemy
import sqlite3

In [11]:
sqlalchemy.create_engine('sqlite:///stores.db')

Engine(sqlite:///stores.db)

In [12]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [13]:
%sql sqlite:///stores.db

'Connected: @stores.db'

In [21]:
%%sql

/* Testing that the connection is working */

SELECT *
FROM customers
LIMIT 5;

 * sqlite:///stores.db
Done.


customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370,21000
112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166,71800
114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300
119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370,118200
121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504,81700


---
### Key questions

Below are the questions we want to answer for this project.

1. Which products should we order more of or less of?
2. How should we tailor marketing and communication strategies to customer behaviors?
3. How much can we spend on acquiring new customers?

#### Question 1: Which products should we order more of or less of?

To answer this question, we want to get an understanding of the low stock items and product performance. The basic idea is that we want to order more of the high peroforming items that are currently low in stock.

To get an indication of low-stock items, we will calculate a low stock score using the formula:

_low stock = SUM(quantityOrdered) / quantityInStock_

We will use product revenue as a measure of product performance:

_product performance = SUM(quantityOrdered * priceEach)_

In [24]:
%%sql

/* Getting the list of low-stock items */

SELECT p.productCode, (od.total_quantity *1.0 / p.quantityInStock) *100.00 AS low_stock
  FROM products p
  JOIN (SELECT productCode, SUM(quantityOrdered) AS total_quantity
  FROM orderdetails
 GROUP BY productCode) od
    ON p.productCode = od.productCode
 ORDER BY low_stock
 LIMIT 10;
 

 * sqlite:///stores.db
Done.


productCode,low_stock
S18_1984,9.383954154727794
S24_3432,9.464323523184417
S32_2206,9.804133751758467
S18_3482,10.02519995617399
S18_1589,10.108383101083831
S12_3380,10.139208593664364
S700_2466,10.193722158914326
S18_2325,10.230917254650416
S12_2823,10.283084925477644
S18_2870,10.47280744732974


In [25]:
%%sql

/* Getting an indication of product performance */
SELECT productCode, SUM(quantityOrdered * priceEach) AS product_perf
  FROM orderdetails
 GROUP BY productCode
 ORDER BY product_perf DESC
 LIMIT 10;

 * sqlite:///stores.db
Done.


productCode,product_perf
S18_3232,276839.98
S12_1108,190755.86
S10_1949,190017.95999999996
S10_4698,170685.99999999997
S12_1099,161531.47999999992
S12_3891,152543.02
S18_1662,144959.90999999997
S18_2238,142530.62999999998
S18_1749,140535.60000000003
S12_2823,135767.03000000003


In [31]:
%%sql

/* Combining the above two steps into one using CTE */
WITH 
lowstockitems AS (
SELECT p.productCode, (od.total_quantity *1.0 / p.quantityInStock) *100.00 AS low_stock
  FROM products p
  JOIN (SELECT productCode, SUM(quantityOrdered) AS total_quantity
          FROM orderdetails
         GROUP BY productCode) od
    ON p.productCode = od.productCode
 ORDER BY low_stock
 LIMIT 10
),
topperfitems AS (
SELECT productCode, SUM(quantityOrdered * priceEach) AS product_perf
  FROM orderdetails
 GROUP BY productCode
 ORDER BY product_perf DESC
)
  
SELECT t.productCode, t.product_perf, p.productName
  FROM topperfitems t
  JOIN products p
    ON t.productCode = p.productCode
 WHERE t.productCode IN (SELECT productCode
                          FROM lowstockitems)
 ORDER BY t.product_perf DESC;

 * sqlite:///stores.db
Done.


productCode,product_perf,productName
S12_2823,135767.03000000003,2002 Suzuki XREO
S18_3482,121890.6,1976 Ford Gran Torino
S18_1984,119050.95,1995 Honda Civic
S18_2325,109992.01000000002,1932 Model A Ford J-Coupe
S18_1589,101778.13000000002,1965 Aston Martin DB5
S18_2870,100770.12,1999 Indy 500 Monte Carlo SS
S12_3380,98718.76000000001,1968 Dodge Charger
S700_2466,89347.79999999999,America West Airlines B757-200
S24_3432,87404.81000000001,2002 Chevy Corvette
S32_2206,33268.76,1982 Ducati 996 R


So, these are the list of high-performing products that are also low in stock. The store should prioritise restocking of these products.

---
#### Question 2: How should we tailor marketing and communication strategies to customer behaviors?

To answer this question, we want to get a sense of which customers are the most profitable for the store. We would want to make sure that we are providing the best customer service and experience to those customers that bring in the most profit, while also trying to engage with the customers who are not as profitable to see whether we could improve sales with them.

In [32]:
%%sql

/* Getting a list of VIPs */
WITH orderprofit AS (
SELECT o.orderNumber, o.orderLineNumber, o.productCode, SUM(o.quantityOrdered * (o.priceEach -p.buyPrice)) AS profit
  FROM orderdetails o
  JOIN products p
    ON o.productCode = p.productCode
 GROUP BY orderNumber
),
custprofitlist AS (
SELECT o.customerNumber, SUM(op.profit) AS profit_from_cust
  FROM orders o
  JOIN orderprofit op
    ON o.orderNumber = op.orderNumber
 GROUP BY o.customerNumber
 ORDER BY profit_from_cust DESC
)

SELECT c.contactLastName, c.contactFirstName, c.city, c.country, cp.profit_from_cust
  FROM customers c
  JOIN custprofitlist cp
    ON c.customerNumber = cp.customerNumber
 ORDER BY cp.profit_from_cust DESC
 LIMIT 5;

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country,profit_from_cust
Freyre,Diego,Madrid,Spain,326519.66000000003
Nelson,Susan,San Rafael,USA,236769.39000000007
Young,Jeff,NYC,USA,72370.09
Ferguson,Peter,Melbourne,Australia,70311.06999999999
Labrune,Janine,Nantes,France,60875.29999999999


In [33]:
%%sql

/* Getting a list of least engaged */
WITH orderprofit AS (
SELECT o.orderNumber, o.orderLineNumber, o.productCode, SUM(o.quantityOrdered * (o.priceEach -p.buyPrice)) AS profit
  FROM orderdetails o
  JOIN products p
    ON o.productCode = p.productCode
 GROUP BY orderNumber
),
custprofitlist AS (
SELECT o.customerNumber, SUM(op.profit) AS profit_from_cust
  FROM orders o
  JOIN orderprofit op
    ON o.orderNumber = op.orderNumber
 GROUP BY o.customerNumber
 ORDER BY profit_from_cust DESC
)

SELECT c.contactLastName, c.contactFirstName, c.city, c.country, cp.profit_from_cust
  FROM customers c
  JOIN custprofitlist cp
    ON c.customerNumber = cp.customerNumber
 ORDER BY cp.profit_from_cust
 LIMIT 5;

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country,profit_from_cust
Young,Mary,Glendale,USA,2610.870000000001
Taylor,Leslie,Brickhaven,USA,6586.0199999999995
Ricotti,Franco,Milan,Italy,9532.93
Schmitt,Carine,Nantes,France,10063.800000000001
Smith,Thomas,London,UK,10868.039999999999


#### Question 3: How much can we spend on acquiring new customers?

To answer how much the store could spend on customer acquisition, knowing how much an additional customer would bring in as profit to the store is important. To make a net positive contribution to the store's profits, the cost to acquire a customer should be less than the profit they will bring.

Below, we will try to get a sense of this by calculating the average profit per customer, so that the store knows the average cost it can spend on acquiring new customers.

In [34]:
%%sql

/* Getting the average profit per customer to help determine the question above */
WITH orderprofit AS (
SELECT o.orderNumber, o.orderLineNumber, o.productCode, SUM(o.quantityOrdered * (o.priceEach -p.buyPrice)) AS profit
  FROM orderdetails o
  JOIN products p
    ON o.productCode = p.productCode
 GROUP BY orderNumber
),
custprofitlist AS (
SELECT o.customerNumber, SUM(op.profit) AS profit_from_cust
  FROM orders o
  JOIN orderprofit op
    ON o.orderNumber = op.orderNumber
 GROUP BY o.customerNumber
 ORDER BY profit_from_cust DESC
)

SELECT AVG(profit_from_cust) AS average_profit_per_cust
  FROM custprofitlist;

 * sqlite:///stores.db
Done.


average_profit_per_cust
39039.5943877551
